## Ejercicio 1. Operaciones con RDD’s

- Crea un RDD con números del 1 al 100. Calcula la cantidad de elementos, la suma total, el promedio y el valor máximo. Imprime los resultados.
- Sobre el RDD anterior, filtra únicamente los números pares. Usa map para transformar cada número filtrado en su cuadrado. Muestra los primeros 10 resultados.
- Transforma el RDD anterior en un RDD de tuplas en formato (categoría, valor), donde la categoría sea el último dígito del cuadrado convertido a string. Utiliza reduceByKey para sumar los valores de cada categoría. Imprime el resultado para cada categoría.


## Ejercicio 2. Operaciones con dataframes
- Sobre los datos de ratings (u.data), películas (u.item) y usuarios (u.user), muestra la película con mayor número de votos positivos (4,5) según los siguientes criterios:
  - Sexo
  - Edad (0-25, 25-45, 45-65, >65)
  - Ocupación

## Ejercicio 3. Análisis de ventas
Disponemos de tres ficheros de ventas de una tienda americana de productos de tecnología, correspondientes a los meses de octubre, noviembre y diciembre. Realiza las siguientes tareas:
- Lectura de los ficheros como RDDs
- Limpieza de datos: eliminar registros vacíos o con un número de campos incorrecto. Detectar y eliminar posibles filas de cabecera (header) si aparecen como registros.
- Convertir a dataframe con el siguiente esquema:
  - "Order ID", Integer
  - "Product", String
  - "Quantity Ordered", Integer
  - "Price Each", Double
  - "Order Date", String
  - "Purchase Address", String
- Unir los registros en un único dataframe "Ventas"
- Generar una tabla temporal "Productos" con los productos vendidos y su precio medio
- Crear campos adicionales en el dataframe Ventas: year, month, state, city, CP a partir de los campos Order Date y Purchase Address
- Obtener el día con mayores ingresos
- Obtener el producto más vendido (por cantidad total) y qué ingresos ha generado en total.
- Listar las 10 ciudades con mayores ventas (en ingresos).
- Tabla de número de pedidos e importe por horas (campo hour extraído de Order Date).
- Almacenar los datos en formato Parquet en la carpeta "sales", particionando por year y month.

## Ejercicio 4. Completar Problema del Viajante con Búsqueda en Anchura en PySpark
Se dispone de un diccionario de distancias entre 10 ciudades españolas. El objetivo es implementar, utilizando PySpark, un algoritmo que resuelva un caso simplificado del Problema del Viajante (TSP). Específicamente, se debe encontrar la ruta de menor distancia que parta desde Madrid y que visite cada una de las ciudades exactamente una vez.

Para ello, se utilizará un recorrido en anchura (Breadth First Search, BFS) sobre un RDD. Cada ruta parcial se representará como una tupla formada por:
- Una lista con el orden de ciudades visitadas.
- La distancia acumulada en esa ruta.

Se empleará una variable broadcast para compartir la tabla de distancias entre los workers, evitando así que cada tarea tenga que reenviar estos datos.

Entender las funciones definidas, completar las llamadas que faltan y ejecutar.

In [0]:
# Diccionario que contiene las distancias (en kilómetros) entre cada par de ciudades.
distancias = {
    "Madrid": {"Barcelona": 620, "Valencia": 350, "Sevilla": 530, "Zaragoza": 320, "Málaga": 530, "Bilbao": 400, "Murcia": 420, "Palma": 500, "Alicante": 420},
    "Barcelona": {"Madrid": 620, "Valencia": 350, "Sevilla": 1000, "Zaragoza": 300, "Málaga": 1080, "Bilbao": 600, "Murcia": 700, "Palma": 210, "Alicante": 520},
    "Valencia": {"Madrid": 350, "Barcelona": 350, "Sevilla": 660, "Zaragoza": 300, "Málaga": 600, "Bilbao": 700, "Murcia": 140, "Palma": 250, "Alicante": 170},
    "Sevilla": {"Madrid": 530, "Barcelona": 1000, "Valencia": 660, "Zaragoza": 700, "Málaga": 210, "Bilbao": 750, "Murcia": 500, "Palma": 650, "Alicante": 450},
    "Zaragoza": {"Madrid": 320, "Barcelona": 300, "Valencia": 300, "Sevilla": 700, "Málaga": 670, "Bilbao": 300, "Murcia": 510, "Palma": 450, "Alicante": 500},
    "Málaga": {"Madrid": 530, "Barcelona": 1080, "Valencia": 600, "Sevilla": 210, "Zaragoza": 670, "Bilbao": 800, "Murcia": 400, "Palma": 700, "Alicante": 480},
    "Bilbao": {"Madrid": 400, "Barcelona": 600, "Valencia": 700, "Sevilla": 750, "Zaragoza": 300, "Málaga": 800, "Murcia": 650, "Palma": 750, "Alicante": 680},
    "Murcia": {"Madrid": 420, "Barcelona": 700, "Valencia": 140, "Sevilla": 500, "Zaragoza": 510, "Málaga": 400, "Bilbao": 650, "Palma": 350, "Alicante": 75},
    "Palma": {"Madrid": 500, "Barcelona": 210, "Valencia": 250, "Sevilla": 650, "Zaragoza": 450, "Málaga": 700, "Bilbao": 750, "Murcia": 350, "Alicante": 520},
    "Alicante": {"Madrid": 420, "Barcelona": 520, "Valencia": 170, "Sevilla": 450, "Zaragoza": 500, "Málaga": 480, "Bilbao": 680, "Murcia": 75, "Palma": 520}
}

# Variable “distancias_compartidas” de solo lectura, distribuida entre nodos del cluster
distancias_compartidas = ## COMPLETAR ##

# Extiende cada ruta parcial añadiendo un nuevo nodo para cada ciudad no visitada.
def expandir_camino(camino):
    ruta = camino[0]
    distancia = camino[1]
    ultimo_nodo = ruta[-1]
    caminos_expandidos = []
    
    nodos_visitados = set(ruta)
    nodos_no_visitados = set(distancias_compartidas.value.keys()) - nodos_visitados
    
    for nodo in nodos_no_visitados:
        nueva_ruta = ruta + [nodo]
        nueva_distancia = distancia + distancias_compartidas.value[ultimo_nodo][nodo]
        caminos_expandidos.append((nueva_ruta, nueva_distancia))
    
    return caminos_expandidos

# Para cada par de caminos, se queda con el de menor distancia
def reducir_caminos(camino1, camino2):
    return camino1 if camino1[1] < camino2[1] else camino2

# Inicializa el camino con el nodo inicial y la distancia 0
nodo_inicial = ['Madrid']
distancia = 0
camino_rdd = ## COMPLETAR ##

# Iterar hasta que todos los nodos sean visitados por todos los caminos
while len(camino_rdd.first()[0]) < len(distancias_compartidas.value):
    camino_rdd = ## COMPLETAR ##

# Ver número de caminos generados
num_caminos = ## COMPLETAR ##
print(f"Se han generado {num_caminos} caminos")

# Obtener el camino más corto
camino_mas_corto = ## COMPLETAR ##

print(f"El camino más corto es: {camino_mas_corto[0]} con una distancia de {camino_mas_corto[1]} km")
